In [86]:
!pip -qq install --upgrade youtube-dl
!pip -qq install git+https://github.com/openai/whisper.git 
!pip -qq install transformers
!pip -qq install sentencepiece
!pip -qq install spacy-transformers

In [87]:
import random
import json 
import pandas as pd
import os
from tqdm import tqdm, tqdm_notebook
import torch
import time
from os.path import exists as path_exists
from pathlib import Path

#Spacy 
import spacy
from spacy.tokens import DocBin
from spacy.util import minibatch
from spacy.training import Example
from spacy.scorer import Scorer
from spacy.lang.es import Spanish
from spacy.pipeline import EntityRuler

#huggingface
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import AutoModelForSeq2SeqLM

#whisper
import whisper

In [88]:
print(os.getcwd())
df = pd.read_csv(r"C:\Users\Admin\Documents\GitHub\anamnese\content\archive.zip", compression='zip', header=0, sep=',', quotechar='"')
df.head()

c:\Users\Admin\Documents\GitHub\anamnese


,Unnamed: 0,description,medical_specialty,sample_name,transcription,keywords
0,0,A 23-year-old white female presents with comp...,Allergy / Immunology,Allergic Rhinitis,"SUBJECTIVE:, This 23-year-old white female pr...","allergy / immunology, allergic rhinitis, aller..."
1,1,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 2,"PAST MEDICAL HISTORY:, He has difficulty climb...","bariatrics, laparoscopic gastric bypass, weigh..."
2,2,Consult for laparoscopic gastric bypass.,Bariatrics,Laparoscopic Gastric Bypass Consult - 1,"HISTORY OF PRESENT ILLNESS: , I have seen ABC ...","bariatrics, laparoscopic gastric bypass, heart..."
3,3,2-D M-Mode. Doppler.,Cardiovascular / Pulmonary,2-D Echocardiogram - 1,"2-D M-MODE: , ,1. Left atrial enlargement wit...","cardiovascular / pulmonary, 2-d m-mode, dopple..."
4,4,2-D Echocardiogram,Cardiovascular / Pulmonary,2-D Echocardiogram - 2,1. The left ventricular cavity size and wall ...,"cardiovascular / pulmonary, 2-d, doppler, echo..."


ICI CHANGER LE HELSINKI MODEL POUR PASSER D L ESPAGNOL AU FRANCAIS §§§§§§§§§§§§§§§§§§§§§§§§§§§§§§   

In [89]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer_t = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-fr")

model_t = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-fr")
  

In [90]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model_t = model_t.to(device)

In [91]:
def translate(text: str):
    inputs = tokenizer_t.encode(text, return_tensors="pt",padding=True,max_length=512,truncation=True)
    inputs = inputs.to(device)
    outputs = model_t.generate(inputs,max_length=512, num_beams=5,early_stopping=True)## num_beams=None ## précédent code 
    return tokenizer_t.decode(outputs[0], skip_special_tokens=True) ## GPT 
    ##translated = tokenizer_t.decode(outputs[0]).replace('',"").replace('',"").replace('',"").strip().lower()

    return translated

In [92]:
df.tail()['description'].apply(translate)


4994    Patient ayant une sinusite sévère il y a envir...
4995    Il s'agit d'un enfant de 14 mois Caucasien qui...
4996    Une femelle pour un physique complet et un sui...
4997              La mère dit qu'il sifflait et toussait.
4998    Réaction allergique aiguë, étiologie incertain...
Name: description, dtype: object

In [93]:
filtered_df = df[['transcription', 'medical_specialty']]
filtered_df.loc[:, 'medical_specialty'] = filtered_df['medical_specialty'].apply(lambda x:str.strip(x))
mask = (filtered_df['medical_specialty'] == 'SOAP / Chart / Progress Notes') | \
       (filtered_df['medical_specialty'] == 'Office Notes') | \
       (filtered_df['medical_specialty'] == 'Consult - History and Phy.') | \
       (filtered_df['medical_specialty'] == 'Emergency Room Reports') | \
       (filtered_df['medical_specialty'] == 'Discharge Summary') | \
       (filtered_df['medical_specialty'] == 'Letters')
filtered_df = filtered_df[~mask]
data_categories  = filtered_df.groupby(filtered_df['medical_specialty'])

filtered_data_categories = data_categories.filter(lambda x:x.shape[0] > 50)
     

In [94]:
filtered_data_categories['medical_specialty'].value_counts()


medical_specialty
Surgery                       1103
Cardiovascular / Pulmonary     372
Orthopedic                     355
Radiology                      273
General Medicine               259
Gastroenterology               230
Neurology                      223
Obstetrics / Gynecology        160
Urology                        158
ENT - Otolaryngology            98
Neurosurgery                    94
Hematology - Oncology           90
Ophthalmology                   83
Nephrology                      81
Pediatrics - Neonatal           70
Pain Management                 62
Psychiatry / Psychology         53
Name: count, dtype: int64

In [95]:
df_cleaned = filtered_data_categories.sample(frac=1.0)
df_cleaned = df_cleaned.dropna(subset=['transcription'])
df_cleaned.head()
     

,transcription,medical_specialty
2874,"CC:, Dysarthria,HX: ,This 52y/o RHF was transf...",Neurology
2123,"EXAM: ,Thoracic Spine.,REASON FOR EXAM: , Inj...",Orthopedic
2328,"PREOPERATIVE DIAGNOSIS: , Recurrent anterior d...",Orthopedic
3533,"REASON FOR VISIT: , Followup of laparoscopic f...",Gastroenterology
1614,"EXAM:, Lexiscan Nuclear Myocardial Perfusion ...",Radiology


Label the NER in English using the huggingface model for biomedical



In [96]:
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForTokenClassification

tokenizer = AutoTokenizer.from_pretrained("d4data/biomedical-ner-all")
model = AutoModelForTokenClassification.from_pretrained("d4data/biomedical-ner-all")

pipe = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple", device=device)
     

In [97]:
tqdm_notebook().pandas()
df_cleaned['ner_en']  = df_cleaned['transcription'].progress_apply(pipe)
     

C:\Users\Admin\AppData\Local\Temp\ipykernel_2692\457871124.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  tqdm_notebook().pandas()


0it [00:00, ?it/s]

  0%|          | 0/3732 [00:00<?, ?it/s]

In [98]:
df_cleaned.head()


,transcription,medical_specialty,ner_en
2874,"CC:, Dysarthria,HX: ,This 52y/o RHF was transf...",Neurology,"[{'entity_group': 'Lab_value', 'score': 0.9996..."
2123,"EXAM: ,Thoracic Spine.,REASON FOR EXAM: , Inj...",Orthopedic,"[{'entity_group': 'Biological_structure', 'sco..."
2328,"PREOPERATIVE DIAGNOSIS: , Recurrent anterior d...",Orthopedic,"[{'entity_group': 'Detailed_description', 'sco..."
3533,"REASON FOR VISIT: , Followup of laparoscopic f...",Gastroenterology,"[{'entity_group': 'Therapeutic_procedure', 'sc..."
1614,"EXAM:, Lexiscan Nuclear Myocardial Perfusion ...",Radiology,"[{'entity_group': 'Diagnostic_procedure', 'sco..."


In [99]:
dict_specialty_keywords = {}
dict_specialty_keywords_fr = {}


def create_dict(list_elem):
    for elem in list_elem:
      if elem['entity_group'] in dict_specialty_keywords:
        dict_specialty_keywords[elem['entity_group']].append(elem['word'])
        dict_specialty_keywords_fr[elem['entity_group']].append(translate(elem['word']))
      else:
        dict_specialty_keywords[elem['entity_group']] = [elem['word']]
        dict_specialty_keywords_fr[elem['entity_group']] = [translate(elem['word'])]
     

In [100]:
tqdm_notebook().pandas()

_ = df_cleaned['ner_en'].progress_apply(create_dict)

C:\Users\Admin\AppData\Local\Temp\ipykernel_2692\4269868783.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  tqdm_notebook().pandas()


0it [00:00, ?it/s]

  0%|          | 0/3732 [00:00<?, ?it/s]

In [101]:
with open("ner_annotations_fr.json", 'w') as f:
    json.dump(dict_specialty_keywords_fr, f) 

In [102]:
tqdm_notebook().pandas()

df_cleaned['transcription_fr'] = df_cleaned['transcription'].progress_apply(translate)

C:\Users\Admin\AppData\Local\Temp\ipykernel_2692\1540691733.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  tqdm_notebook().pandas()


0it [00:00, ?it/s]

  0%|          | 0/3732 [00:00<?, ?it/s]

In [103]:
df_cleaned.to_csv("df_cleaned.csv")


Create the annotated dataset


In [105]:
dict_specialty_keywords_fr = json.load(open("ner_annotations_fr.json"))

dict_specialty_keywords_fr.keys()

dict_keys(['Lab_value', 'Diagnostic_procedure', 'Disease_disorder', 'Clinical_event', 'Nonbiological_location', 'Date', 'Detailed_description', 'Sign_symptom', 'Subject', 'Activity', 'Biological_structure', 'Severity', 'Duration', 'Area', 'Therapeutic_procedure', 'Medication', 'Time', 'Age', 'Frequency', 'Dosage', 'Administration', 'Sex', 'History', 'Other_entity', 'Distance', 'Personal_background', 'Coreference', 'Volume', 'Quantitative_concept', 'Color', 'Family_history', 'Shape', 'Occupation', 'Other_event', 'Qualitative_concept', 'Outcome', 'Texture', 'Height'])

In [106]:

for k in dict_specialty_keywords_fr.keys():
  dict_specialty_keywords_fr[k] = set(dict_specialty_keywords_fr[k])

In [107]:
for k in dict_specialty_keywords_fr.keys():
  dict_specialty_keywords_fr[k] = [el.strip() for el in list(dict_specialty_keywords_fr[k]) if len(el.strip())>=4 or k=="Age"]
     


And delete some wrong words:

In [116]:
for category in dict_specialty_keywords_fr.keys():
    print(category)

Lab_value
Diagnostic_procedure
Disease_disorder
Clinical_event
Nonbiological_location
Date
Detailed_description
Sign_symptom
Subject
Activity
Biological_structure
Severity
Duration
Area
Therapeutic_procedure
Medication
Time
Age
Frequency
Dosage
Administration
Sex
History
Other_entity
Distance
Personal_background
Coreference
Volume
Quantitative_concept
Color
Family_history
Shape
Occupation
Other_event
Qualitative_concept
Outcome
Texture
Height


In [112]:
#for the class sex we delete the following words
list_to_delete = ['vétéran', 'dentiste', 'mariés',]

for el in list_to_delete: 
  dict_specialty_keywords_fr['Sex'].pop(dict_specialty_keywords_fr['Sex'].index(el))

In [117]:
dict_specialty_keywords_fr['History']

['divorce',
 'année',
 "ne pas fumer, boire de l'alcool",
 'Bien soigné',
 "de mariage ou d'enfants",
 "pas d'histoire",
 'néphropathie diabétique',
 'glioblastome',
 'Activités sexuelles',
 'prothèses dentaires',
 'refusé',
 'coronaire',
 'multiples',
 'venin',
 '##lux',
 'Voyages ou expositions récents',
 'toute intervention médicale antérieure importante',
 'bière par jour',
 'allergique à la codéine, à la pénicilline',
 'DIRECTION',
 '##se',
 'nourris',
 'consommer rarement',
 '# #Ropop',
 'collège',
 'glissé',
 'foie positif',
 '##Nchospasme',
 'Très bien.',
 'diabète sucré',
 "pas d' antécédents de tuberculose",
 'lourd',
 '8ème',
 'vertiges',
 'arrêter',
 'hernies',
 'médicaments',
 'fumées',
 'Nous',
 'colorant',
 'plus jamais',
 'ourlet',
 'de parkinson',
 '- semaine',
 'hystérectomie totale',
 '25 ans',
 'La tension',
 'mules',
 'colectomie',
 'non-fumeur',
 'morbide',
 'le tabac refusé',
 'une bicyclette',
 'houille',
 "antécédents de l'artère coronaire",
 'gravida',
 'malad

In [121]:
#for the class History we delete the following words
list_to_delete =[ '25 ans']

for el in list_to_delete: 
  dict_specialty_keywords_fr['History'].pop(dict_specialty_keywords_fr['History'].index(el))
     

In [131]:
#for the class Nonbiological_location we delete the following words
list_to_delete = [ 'Dr X']
for el in list_to_delete: 
  dict_specialty_keywords_fr['Nonbiological_location'].pop(dict_specialty_keywords_fr['Nonbiological_location'].index(el))
     


ValueError: 'Dr X' is not in list

In [132]:
# for the class Age
list_to_delete = ['seul', 'moyen', 'et', ',', 'à', 'kg', '1812 g', 'école', 'regardant', '## poids', 'ou']

for el in list_to_delete: 
  try:
    dict_specialty_keywords_fr['Age'].pop(dict_specialty_keywords_fr['Age'].index(el))
  except:
    continue

In [133]:
list_to_delete = ['Quel est le sens de la bible ?', 'admission', 'vieux', 'couple', 'autres', 'quelques', 'peu après',
 'dedans', 'autres', 'âge 92', 'peut', 'un jour hors de l’école', 'nuit', 'âge de 12 ans', 'les premiers 4', '8 semaines d’âge gestationnel', 'jour de',
 'âge de 42 ans', 'peut', 'cette fois', 'plusieurs', '6 ans', 'vieux', 'âge de 75 ans', '37.5 semaines de gestation', '20 ans', '20 ans', '89 ans', '31 mois', 'vie', 'plus tard dans la journée', 'presque', 'poste', 'certains', 'Quel est le problème ?', 'ceci', 'fait']

for el in list_to_delete: 
  try:
    dict_specialty_keywords_fr['Date'].pop(dict_specialty_keywords_fr['Date'].index(el))
  except:
    continue

In [134]:
list_to_delete = ['riche', 'acquiescer', 'tubes', 'L4 - 5', 'voix réelle', 'multiple', 'arrière', 'oui, bien sûr.', 'oui, bien sûr.',
 'processus', 'jambon', '- L4', 'tapis', 'et aînés', 'ajouter', 'courroie', 'court', 'chaque', 'armes',
 'pour', 'droits de l’homme', 'péché', 'basilic', 'et chants latéraux', 'un point', 'L1 - 2', 'faux',
 'repos', 'gomme à mâcher', 'dans le', 'peu', 'bien', 'entre', 'de la', 'un', 'Comme']

for el in list_to_delete: 
  try:
    dict_specialty_keywords_fr['Biological_structure'].pop(dict_specialty_keywords_fr['Biological_structure'].index(el))
  except:
    continue
     

In [135]:
list_to_delete = ['dehors !', 'carte', 'dans le', 'solution saline', 'chapitre', 'sombre', 'nuit', 'coton', 'en vrac', 'clair',
 'recherche de mots', 'travaillé', 'dehors', 'photo', 'capable de', 'respiration', 'frappé', 'pertes sur le terrain',
 'tournoyant', 'l’a fait', 'stabilisé', 'L5 -', 'niveau', 'navire', 'force', 'voyage', 'chair', 'fonction',
 'architecture du sommeil', 'jaune', 'actif', 'signal de câble', 'journal', 'dentelle', 'résoudre', 'talc']

for el in list_to_delete: 
  try:
    dict_specialty_keywords_fr['Sign_symptom'].pop(dict_specialty_keywords_fr['Sign_symptom'].index(el))
  except:
    continue

In [136]:
list_to_delete = ['peut', 'difficile', 'moins', 'partie', 'comme', 'premier']

for el in list_to_delete: 
  try:
    dict_specialty_keywords_fr['Detailed_description'].pop(dict_specialty_keywords_fr['Detailed_description'].index(el))
  except:
    continue



In [137]:
list_to_delete = ['Dans le', 'peut', 'a', 'lieu']

for el in list_to_delete: 
  try:
    dict_specialty_keywords_fr['Therapeutic_procedure'].pop(dict_specialty_keywords_fr['Therapeutic_procedure'].index(el))
  except:
    continue

# dict_specialty_keywords_fr['Therapeutic_procedure']

In [138]:

list_to_delete =['pour', 'vivant', 'trois', '- oui.', 'cultivé', 'un 18', 'sept livres', 'carte', 'chambre', 'plus âgé', 'aînés',
                 'âge', 'approuvées', 'début', 'peu', 'fertile', 'quatrième']

for el in list_to_delete: 
  try:
    dict_specialty_keywords_fr['Subject'].pop(dict_specialty_keywords_fr['Subject'].index(el))
  except:
    continue


Now, we delete some keys

In [ ]:

list_delete = ['Frequency', 'Quantitative_concept',  'Coreference', 'Distance', 'Time', 'Area','Administration',
               'Other_entity', 'Volume', 'Shape', 'Texture', 'Other_event', 'Height','Lab_value', 'Duration']

for elem in list_delete:
  del dict_specialty_keywords_fr[elem] 

Create the ruler ner model



In [139]:
class RulerModel():
    def __init__(self, dict_specialty_keywords):
        self.ruler_model = spacy.blank('es')
        self.entity_ruler =  self.ruler_model.add_pipe('entity_ruler')
        
        total_patterns = []

        for k,v in dict_specialty_keywords_fr.items():
        
          patterns = self.create_patterns(v, k)
          total_patterns.extend(patterns)
       
        self.add_patterns_into_ruler(total_patterns)
        
       
    def create_patterns(self, entity_type_set, entity_type):
        patterns = []
        for item in entity_type_set:
            pattern = {'label': entity_type, 'pattern': item}
            patterns.append(pattern)
        return patterns
    
    def add_patterns_into_ruler(self, total_patterns):
        self.entity_ruler.add_patterns(total_patterns)


and the dataset generator



In [140]:
class GenerateDataset(object):
    
    def __init__(self, ruler_model):
        self.ruler_model = ruler_model.ruler_model
        
    def find_entitytypes(self, text):
        ents = [] 
        doc = self.ruler_model(str(text))
        for ent in doc.ents:
            ents.append((ent.start_char, ent.end_char, ent.label_))
        return ents     
    
    def assign_labels_to_documents(self, df):
        dataset = []
        text_list = df['transcription_fr'].values.tolist()
        for text in text_list:
            ents = self.find_entitytypes(text)
            if len(ents) > 0:
                dataset.append((text, {'entities': ents}))
            else:
                continue 
        return dataset
     

In [143]:
print(df_cleaned.columns)

Index(['Unnamed: 0', 'description', 'medical_specialty', 'sample_name',
       'transcription', 'keywords'],
      dtype='object')


In [144]:
ruler_ner = RulerModel(dict_specialty_keywords_fr)
ds_ner = GenerateDataset(ruler_ner)
df_cleaned = pd.read_csv(r"C:\Users\Admin\Documents\GitHub\anamnese\content\archive.zip", compression='zip', header=0, sep=',', quotechar='"')## GPT
df_cleaned.rename(columns={'transcription': 'transcription_fr'}, inplace=True)

df_ner = ds_ner.assign_labels_to_documents(df_cleaned)


Let's check the dataset generated:



In [145]:
df_ner[0]


('SUBJECTIVE:,  This 23-year-old white female presents with complaint of allergies.  She used to have allergies when she lived in Seattle but she thinks they are worse here.  In the past, she has tried Claritin, and Zyrtec.  Both worked for short time but then seemed to lose effectiveness.  She has used Allegra also.  She used that last summer and she began using it again two weeks ago.  It does not appear to be working very well.  She has used over-the-counter sprays but no prescription nasal sprays.  She does have asthma but doest not require daily medication for this and does not think it is flaring up.,MEDICATIONS: , Her only medication currently is Ortho Tri-Cyclen and the Allegra.,ALLERGIES: , She has no known medicine allergies.,OBJECTIVE:,Vitals:  Weight was 130 pounds and blood pressure 124/78.,HEENT:  Her throat was mildly erythematous without exudate.  Nasal mucosa was erythematous and swollen.  Only clear drainage was seen.  TMs were clear.,Neck:  Supple without adenopathy.

Training the Spacy NER model
Frist, split in train - test



In [146]:
random.shuffle(df_ner)

train_data = df_ner[:int(len(df_ner)*0.8)]
test_data = df_ner[int(len(df_ner)*0.8):]

And we create the binary dataset as spacy required
to generate train file:



In [147]:
db = DocBin() # create a DocBin object

nlp = spacy.blank('fr') 

for text, annot in tqdm(test_data): # data in previous format
    doc = nlp.make_doc(text) # create doc object from text
    ents = []
    for start, end, label in annot["entities"]: # add character indexes
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents # label the text with the ents
    db.add(doc)

db.to_disk("./test.spacy") # save the docbin object

100%|██████████| 982/982 [00:11<00:00, 86.22it/s] 


to generate test file:


In [148]:
db = DocBin() # create a DocBin object

nlp = spacy.blank('es') 

for text, annot in tqdm(test_data): # data in previous format
    doc = nlp.make_doc(text) # create doc object from text
    ents = []
    for start, end, label in annot["entities"]: # add character indexes
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents # label the text with the ents
    db.add(doc)

db.to_disk("./test.spacy") # save the docbin object

100%|██████████| 982/982 [00:09<00:00, 107.57it/s]


Go to spacy and generate the case config file, as we show in the image below:
download that base_config file and place in the same diretory



In [155]:
#!python -m spacy init fill-config base_config.cfg config.cfg
!python -m spacy init fill-config base_config-cpu.cfg config.cfg


Usage: python -m spacy init fill-config [OPTIONS] BASE_PATH [OUTPUT_FILE]
Try 'python -m spacy init fill-config --help' for help.

Error: Invalid value for 'BASE_PATH': File 'base_config-cpu.cfg' does not exist.


✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy

Finally Train the NER model


     ---------------------------------------- 0.0/573.3 MB ? eta -:--:--
     ---------------------------------------- 0.0/573.3 MB ? eta -:--:--
     -------------------------------------- 0.0/573.3 MB 217.9 kB/s eta 0:43:52
     -------------------------------------- 0.0/573.3 MB 245.8 kB/s eta 0:38:53
     -------------------------------------- 0.1/573.3 MB 476.3 kB/s eta 0:20:04
     ---------------------------------------- 1.4/573.3 MB 6.3 MB/s eta 0:01:31
     --------------------------------------- 4.8/573.3 MB 17.9 MB/s eta 0:00:32
      -------------------------------------- 8.7/573.3 MB 27.7 MB/s eta 0:00:21
      ------------------------------------- 12.1/573.3 MB 81.8 MB/s eta 0:00:07
     - ------------------------------------ 15.9/573.3 MB 72.6 MB/s eta 0:00:08
     - ------------------------------------ 19.5/573.3 MB 81.8 MB/s eta 0:00:07
     - ------------------------------------ 23.5/573.3 MB 93.0 MB/s eta 0:00:06
     - ------------------------------------ 26.1/573.3

  error: subprocess-exited-with-error
  
  × pip subprocess to install build dependencies did not run successfully.
  │ exit code: 1
  ╰─> [498 lines of output]
        Using cached setuptools-69.0.2-py3-none-any.whl.metadata (6.3 kB)
        Using cached cymem-2.0.8-cp311-cp311-win_amd64.whl.metadata (8.6 kB)
        Using cached preshed-3.0.9-cp311-cp311-win_amd64.whl.metadata (2.2 kB)
        Using cached murmurhash-1.0.10-cp311-cp311-win_amd64.whl.metadata (2.0 kB)
           ---------------------------------------- 0.0/189.7 kB ? eta -:--:--
           ---------------------------------------- 0.0/189.7 kB ? eta -:--:--
           -- ------------------------------------- 10.2/189.7 kB ? eta -:--:--
           ----------- ------------------------- 61.4/189.7 kB 812.7 kB/s eta 0:00:01
           ------------------------------------ - 184.3/189.7 kB 2.2 MB/s eta 0:00:01
           -------------------------------------- 189.7/189.7 kB 1.4 MB/s eta 0:00:00
        Installing build depen

In [156]:
!python -m spacy train config.cfg --output ./output --paths.train ./train.spacy --paths.dev ./test.spacy 
#--gpu-id 0


ℹ Saving to output directory: output
ℹ Using CPU

=========================== Initializing pipeline ===========================


Traceback (most recent call last):
  File "c:\Users\Admin\anaconda3\Lib\site-packages\spacy\language.py", line 1327, in initialize
    init_vocab(
  File "c:\Users\Admin\anaconda3\Lib\site-packages\spacy\training\initialize.py", line 142, in init_vocab
    load_vectors_into_model(nlp, vectors)
  File "c:\Users\Admin\anaconda3\Lib\site-packages\spacy\training\initialize.py", line 164, in load_vectors_into_model
    vectors_nlp = load_model(name, vocab=nlp.vocab, exclude=exclude)
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Admin\anaconda3\Lib\site-packages\spacy\util.py", line 472, in load_model
    raise IOError(Errors.E050.format(name=name))
OSError: [E050] Can't find model 'fr_core_news_lg'. It doesn't seem to be a Python package or a valid path to a data directory.

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen r

Integrating the Whisper ASR


In [ ]:
if not path_exists('transcripts'):
    !mkdir transcripts

In [ ]:
YouTubeID = 'xOZM-1p-jAk' # ¿Qué es la enfermedad de Crohn y cómo detectarla?
OutputFile = 'test_audio_youtube_5.m4a'

In [ ]:
if not path_exists(OutputFile):
    !youtube-dl -o 
YouTubeID --extract-audio --restrict-filenames -f 'bestaudio[ext=m4a]'
   

In [ ]:
model_whisper = whisper.load_model("small")
result = model_whisper.transcribe(OutputFile)
print(result["text"])

In [ ]:
import spacy_transformers
nlp1 = spacy.load(r"/content/output/model-best") #load the best model

doc = nlp1(result["text"])

In [ ]:
ents = dict_specialty_keywords_fr.keys()


import random
random.seed(38)

colors = {}
for j in ents:
    rand_colors = ["#"+''.join([random.choice('ABCDEF0123456789') for i in range(6)])]
    colors[j] = rand_colors[0]

options = {"ents": list(ents), "colors": colors }

In [ ]:
spacy.displacy.render(doc, style='ent',jupyter=True, options=options)
